# Install requirements 

In [2]:
# Install requirements
!pip install fastapi==0.68.1
!pip install opencv-python==4.5.3.56
# !pip install Pillow==8.3.2
!pip install timm==0.4.12
!pip install python-multipart==0.0.5
!pip install uvicorn==0.15.0

In [3]:
!pip install nest-asyncio

In [4]:
!pip install pyngrok
!pip install aiofiles

In [1]:
from fastapi import FastAPI, File, UploadFile, Request
from fastapi.responses import HTMLResponse, StreamingResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import io
from io import BytesIO
import json
import datetime
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

import time
import os
import copy
import random

import uvicorn
import base64
from base64 import encodebytes
from typing import List, Optional

# Connect to Google Drive

In [5]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# create a symbolic link
!ln -s /content/gdrive/My\ Drive/ /mydrive

!ls /mydrive

# navigate to /mydrive/car-damage-project
%cd /mydrive/car-damage-project
!ls

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 Bewerbungsmaterial		'Introduction to ML'   yolo_pictures
 car-damage-project		'My Drive'	       yolov4
'Colab Notebooks'		'USA 2019'	       yolov4_demo
'Fotos Powderfest Flumserberg'	'UZH Colaboration'     YOLOv4-training.ipynb
/content/gdrive/My Drive/car-damage-project
car-damage-detection-notebooks	models	   test_image.JPEG
gen_paths.py			static	   testpaths.txt
kaggle_data			templates  trainpaths.txt


## Model Loading

In [6]:
%cd /mydrive/yolov4/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/gdrive/My Drive/yolov4/darknet


In [7]:
# build darknet 
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [8]:
%cd /mydrive/car-damage-project

/content/gdrive/My Drive/car-damage-project


In [10]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [11]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    input_size = (224, 224)

    return model_ft, input_size

In [21]:
def load_model(path_to_model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_name = 'densenet'
    loaded_model, _ = initialize_model(model_name, num_classes=2, feature_extract=False, use_pretrained=True)
    loaded_model.load_state_dict(torch.load(path_to_model)) # it takes the loaded dictionary, not the path file itself
    loaded_model.eval()
    return loaded_model

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])
    img = Image.open(io.BytesIO(image_bytes))
    return my_transforms(img).unsqueeze(0)

def get_prediction(image_bytes):
    Name_dict = {
    0: 'damaged',
    1: 'not damaged'}

    tensor = transform_image(image_bytes=image_bytes)
    prediction = model(tensor)
    _ , pred = torch.max(prediction, 1)
    name = Name_dict[pred.item()]
    return str(name)


def get_result(image_file, filename):
    test_file = open('/content/test.txt', 'w')
    test_file.write('/mydrive/car-damage-project/demo_images/'+filename+'\n')
    test_file.close()

    start_time = datetime.datetime.now()
    image_bytes = image_file.file.read()
    name = get_prediction(image_bytes)
    end_time = datetime.datetime.now()
    time_diff = (end_time - start_time)
    execution_time = f'{round(time_diff.total_seconds() * 1000)} ms'

    if name == "damaged": 
        %cd /mydrive/yolov4/darknet

        #!./darknet detector test data/obj.data cfg/yolov4-custom-testing.cfg /mydrive/yolov4/training/yolov4-custom_best.weights -dont_show </content/test.txt>

        pil_img = Image.open('predictions.jpg', mode='r') # reads the PIL image
        byte_arr = io.BytesIO()
        pil_img.save(byte_arr, format='PNG') # convert the PIL image to byte array
        bs64 = encodebytes(byte_arr.getvalue()).decode('utf-8')  # encode as base64
        %cd /mydrive/car-damage-project
    else:
        encoded_string = base64.b64encode(image_bytes)
        bs64 = encoded_string.decode('utf-8')
    result =  {
      "inference_time":execution_time,
      "image_data": f'data:image/jpeg;base64, {bs64}',
      "prediction":{
          "name":name    
       } 
    }
    return result

# Setup the server

In [15]:
FILE = "/mydrive/car-damage-project/models/densenet_pretrained_lrs.pth"
model = load_model(FILE)

app = FastAPI()

app.mount("/static", StaticFiles(directory="static"))
templates=Jinja2Templates(directory="templates")

@app.get("/")
async def main(request:Request):
  return templates.TemplateResponse("index.html", {"request":request})

@app.post("/")
async def home_predict(request:Request, file:UploadFile = File(...)):
  result = get_result(image_file = file, filename = file.filename)
  return templates.TemplateResponse("index.html", {"request":request, "result":result})

@app.post("/predict")
async def predict(file:UploadFile = File(...)):
  return get_result(image_file = file)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [16]:
from pyngrok import ngrok
auth_token = "24JkhRzGyo5sXizFtiIYKvWJG7j_6K2ngnocxRXXrBEt18a8N" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

os.system(f"ngrok authtoken {auth_token}")

0

In [17]:
# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)

In [18]:
# Check if it exists
!ps aux | grep ngrok

root        3181  3.0  0.1 726908 24520 ?        Sl   21:42   0:00 /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root        3191  3.0  0.0  39200  6504 ?        S    21:42   0:00 /bin/bash -c ps aux | grep ngrok
root        3193  0.0  0.0  38576  5636 ?        S    21:42   0:00 grep ngrok


# Run FastAPI app with uvicorn

In [19]:
import nest_asyncio
nest_asyncio.apply()

In [22]:
# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

NgrokTunnel: "https://c927-34-82-186-44.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [1929]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     212.40.1.5:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:67c:10ec:574f:8000::d3:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:67c:10ec:574f:8000::d3:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1929]


In [23]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)